### Installation and importing client ids

In [19]:
!pip install spotipy

  Using cached spotipy-2.22.1-py3-none-any.whl (28 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 kB 2.6 MB/s eta 0:00:00a 0:00:01
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)


In [1]:
!pip install torch

  Using cached torch-2.0.0-cp311-none-macosx_10_9_x86_64.whl (139.5 MB)


In [5]:
import os
import requests
import torch
#import torchaudio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [61]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
cid = '49789426e6c04428a2befbd3c2d79b02'
secret = 'd18032252a0d4634aa8ca21356894e79'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

#### Import the existing csv files as dataframes

In [15]:
danceable_tracks_df = pd.read_csv('/Users/cheffbcookin/Desktop/EC523/Song_Feature_Pred/Sample_Data/danceable_tracks-1.csv')
non_danceable_tracks_df = pd.read_csv('/Users/cheffbcookin/Desktop/EC523/Song_Feature_Pred/Sample_Data/non-danceable_tracks-1.csv')
instrumental_tracks_df = pd.read_csv('/Users/cheffbcookin/Desktop/EC523/Song_Feature_Pred/Sample_Data/instrumental_tracks-1.csv')
non_instrumental_tracks_df = pd.read_csv('/Users/cheffbcookin/Desktop/EC523/Song_Feature_Pred/Sample_Data/non-instrumental_tracks-1.csv')
speechy_tracks_df = pd.read_csv('/Users/cheffbcookin/Desktop/EC523/Song_Feature_Pred/Sample_Data/speechy_tracks-1.csv')
non_speechy_tracks_df = pd.read_csv('/Users/cheffbcookin/Desktop/EC523/Song_Feature_Pred/Sample_Data/non-speechy_tracks-1.csv')
acoustic_tracks_df = pd.read_csv('/Users/cheffbcookin/Desktop/EC523/Song_Feature_Pred/Sample_Data/acoustic_tracks-1.csv')
non_acoustic_tracks_df = pd.read_csv('/Users/cheffbcookin/Desktop/EC523/Song_Feature_Pred/Sample_Data/non-acoustic_tracks-1.csv')
energetic_tracks_df = pd.read_csv('/Users/cheffbcookin/Desktop/EC523/Song_Feature_Pred/Sample_Data/energetic_tracks-1.csv')
non_energetic_tracks_df = pd.read_csv('/Users/cheffbcookin/Desktop/EC523/Song_Feature_Pred/Sample_Data/non-energetic_tracks-1.csv')
all_tracks_df = pd.read_csv('/Users/cheffbcookin/Desktop/EC523/Song_Feature_Pred/Sample_Data/all_tracks-1.csv')

In [16]:
dataframes = {
    'Danceable Tracks': danceable_tracks_df,
    'Non-danceable Tracks': non_danceable_tracks_df,
    'Instrumental Tracks': instrumental_tracks_df,
    'Non-instrumental Tracks': non_instrumental_tracks_df,
    'Speechy Tracks': speechy_tracks_df,
    'Non-speechy Tracks': non_speechy_tracks_df,
    'Acoustic Tracks': acoustic_tracks_df,
    'Non-acoustic Tracks': non_acoustic_tracks_df,
    'Energetic Tracks': energetic_tracks_df,
    'Non-energetic Tracks': non_energetic_tracks_df,
    'All Tracks': all_tracks_df
}

##### Initialize dictionary to hold list of tracks with specific features

We will later save this dictionary and reuse it 

In [36]:
# Load existing DataFrame from CSV file (if it exists)
try:
    tracks_data = pd.read_csv('tracks_data.csv')
except FileNotFoundError:
    tracks_data = pd.DataFrame(columns=['track_id', 'track_name', 'track_artist', 'preview_url'])

In [43]:
# Define danceability range and step size
start_danceability = 0.7
end_danceability = 1.0
step_size = 0.001

# Iterate over danceability values and perform searches
for danceability in reversed(range(int(start_danceability / step_size), int(end_danceability / step_size))):
    # Define search query
    query = f'danceability:{danceability*step_size:.2f}'

    # Search for tracks
    results = sp.search(q=query, type='track', market='US', limit=50)

    # Add new tracks to tracks_data DataFrame (if not already in DataFrame)
    for track in results['tracks']['items']:
        if track['id'] is not None and track['id'] not in tracks_data['track_id'].values:
            track_name = track['name']
            track_artist = track['artists'][0]['name']
            preview_url = track['preview_url']
            tracks_data = tracks_data.append({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, ignore_index=True)

# Print number of tracks found and write to CSV file
print(f'Found {len(tracks_data)} tracks')

# Print tracks_data DataFrame
print(tracks_data.head())

/var/folders/gd/pxcx2w0x59j2bsjbzhy62nvw0000gn/T/ipykernel_5173/1520926489.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tracks_data = tracks_data.append({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, ignore_index=True)
/var/folders/gd/pxcx2w0x59j2bsjbzhy62nvw0000gn/T/ipykernel_5173/1520926489.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tracks_data = tracks_data.append({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, ignore_index=True)
/var/folders/gd/pxcx2w0x59j2bsjbzhy62nvw0000gn/T/ipykernel_5173/1520926489.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tracks_data = tracks_data.app

Found 379 tracks
                 track_id          track_name  track_artist  \
0  4JRIH7oRNb2OyK5FtW6LrU           Breakable       Dabaill   
1  7c6CyAPPp4Lq9vJ00LTi5R          Dance Baby  Michael Ward   
2  5ssQlsPUmOqvN8788Mjddu  Dance Lovers - 4/4     Ballerina   
3  405GG3OKeb6xsFQsXpJIL2          Dance Life      Puteroxy   
4  3nAstOmd0ZxExayGe6QxN1     She's Dangerous  Tom Tom Club   

                                         preview_url  
0  https://p.scdn.co/mp3-preview/93cf917c0fda88db...  
1  https://p.scdn.co/mp3-preview/6b2082270aef17b3...  
2  https://p.scdn.co/mp3-preview/99883ba697f0e723...  
3  https://p.scdn.co/mp3-preview/cc0f48017b69c1be...  
4  https://p.scdn.co/mp3-preview/1f64f384fbe53208...  


In [46]:
# We will save the data to a CSV file so that we can use it later

tracks_data.to_csv('tracks_data.csv', index=False)
print(f'Found {len(tracks_data)} tracks')

Found 1399 tracks


##### We did it for danceability as test feature. Now we do it for all features

In [63]:
# load the data from the CSV file
tracks_data = pd.read_csv('tracks_data_plus.csv')
print(f'Found {len(tracks_data)} tracks')

Found 15371 tracks


In [ ]:
# Define feature range and step size
start = 0.0
end = 1.0
step_size = 0.005

# Iterate over danceability values and perform searches
for feature in reversed(range(int(start / step_size), int(end / step_size))):
    # Define search query
    print(f'working with feature interval: {feature*step_size:.3f}')
    query = f'danceability:{feature*step_size:.3f}'

    # Search for tracks
    results = sp.search(q=query, type='track', market='US', limit=50)

    # Add new tracks to tracks_data DataFrame (if not already in DataFrame)
    for track in results['tracks']['items']:
        if track['id']:
            if track['id'] is not None and track['id'] not in tracks_data['track_id'].values:
                track_name = track['name']
                track_artist = track['artists'][0]['name']
                preview_url = track['preview_url']
                tracks_data = tracks_data.append({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, ignore_index=True)

    # Print number of tracks found and write to CSV file
    print(f'Found {len(tracks_data)} tracks')
    tracks_data.to_csv('tracks_data.csv', index=False)
    
    # Define search query
    query = f'instrumentalness:{feature*step_size:.3f}'

    # Search for tracks
    results = sp.search(q=query, type='track', market='US', limit=50)

    # Add new tracks to tracks_data DataFrame (if not already in DataFrame)
    for track in results['tracks']['items']:
        if track['id']:
            track_id = track['id']
            if track_id not in tracks_data['track_id'].values:
                track_name = track['name']
                track_artist = track['artists'][0]['name']
                preview_url = track['preview_url']
                tracks_data = tracks_data.append({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, ignore_index=True)

    # Print number of tracks found and write to CSV file
    print(f'Found {len(tracks_data)} tracks')
    tracks_data.to_csv('tracks_data.csv', index=False)
    
    # Define search query
    query = f'speechiness:{feature*step_size:.3f}'

    # Search for tracks
    results = sp.search(q=query, type='track', market='US', limit=50)

    # Add new tracks to tracks_data DataFrame (if not already in DataFrame)
    for track in results['tracks']['items']:
        if track['id']:
            if track['id'] is not None and track['id'] not in tracks_data['track_id'].values:
                track_name = track['name']
                track_artist = track['artists'][0]['name']
                preview_url = track['preview_url']
                tracks_data = tracks_data.append({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, ignore_index=True)

    # Print number of tracks found and write to CSV file
    print(f'Found {len(tracks_data)} tracks')
    tracks_data.to_csv('tracks_data.csv', index=False)
    
    # Define search query
    query = f'acousticness:{feature*step_size:.3f}'

    # Search for tracks
    results = sp.search(q=query, type='track', market='US', limit=50)

    # Add new tracks to tracks_data DataFrame (if not already in DataFrame)
    for track in results['tracks']['items']:
        if track['id']:
            if track['id'] is not None and track['id'] not in tracks_data['track_id'].values:
                track_name = track['name']
                track_artist = track['artists'][0]['name']
                preview_url = track['preview_url']
                tracks_data = tracks_data.append({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, ignore_index=True)

    # Print number of tracks found and write to CSV file
    print(f'Found {len(tracks_data)} tracks')
    tracks_data.to_csv('tracks_data.csv', index=False)
    
    # Define search query
    query = f'energy:{feature*step_size:.3f}'

    # Search for tracks
    results = sp.search(q=query, type='track', market='US', limit=50)

    # Add new tracks to tracks_data DataFrame (if not already in DataFrame)
    for track in results['tracks']['items']:
        if track['id']:
            if track['id'] is not None and track['id'] not in tracks_data['track_id'].values:
                track_name = track['name']
                track_artist = track['artists'][0]['name']
                preview_url = track['preview_url']
                tracks_data = tracks_data.append({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, ignore_index=True)

    # Print number of tracks found and write to CSV file
    print(f'Found {len(tracks_data)} tracks')
    tracks_data.to_csv('tracks_data.csv', index=False)

### THIS IS ONLY IF WE NEED EVEN MORE DATA

In [65]:
# Define feature range and step size
start = 0.0
end = 1.0
step_size = 0.001

# Iterate over danceability values and perform searches
for feature in reversed(range(int(start / step_size), int(end / step_size))):
    # Define search query
    print(f'working with feature interval: {feature*step_size:.3f}')
    query = f'danceability:{feature*step_size:.3f}'

    # Search for tracks
    results = sp.search(q=query, type='track', market='US', limit=50)

    # Add new tracks to tracks_data DataFrame (if not already in DataFrame)
    for track in results['tracks']['items']:
        if track['id'] is not None and track['id'] not in tracks_data['track_id'].values:
            track_name = track['name']
            track_artist = track['artists'][0]['name']
            preview_url = track['preview_url']
            df_temp = pd.DataFrame({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, index=[0])
            #tracks_data = tracks_data.append({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, ignore_index=True)
            tracks_data = pd.concat([tracks_data, df_temp], axis=0)
            
    # Print number of tracks found and write to CSV file
    print(f'Found {len(tracks_data)} tracks')
    tracks_data.to_csv('tracks_data_plus.csv', index=False)
    
    # Define search query
    query = f'instrumentalness:{feature*step_size:.3f}'

    # Search for tracks
    results = sp.search(q=query, type='track', market='US', limit=50)

    # Add new tracks to tracks_data DataFrame (if not already in DataFrame)
    for track in results['tracks']['items']:
        if track['id'] is not None and track['id'] not in tracks_data['track_id'].values:
            track_name = track['name']
            track_artist = track['artists'][0]['name']
            preview_url = track['preview_url']
            df_temp = pd.DataFrame({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, index=[0])
            #tracks_data = tracks_data.append({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, ignore_index=True)
            tracks_data = pd.concat([tracks_data, df_temp], axis=0)
    # Print number of tracks found and write to CSV file
    print(f'Found {len(tracks_data)} tracks')
    tracks_data.to_csv('tracks_data_plus.csv', index=False)
    
    # Define search query
    query = f'speechiness:{feature*step_size:.3f}'

    # Search for tracks
    results = sp.search(q=query, type='track', market='US', limit=50)

    # Add new tracks to tracks_data DataFrame (if not already in DataFrame)
    for track in results['tracks']['items']:
        if track['id'] is not None and track['id'] not in tracks_data['track_id'].values:
            track_name = track['name']
            track_artist = track['artists'][0]['name']
            preview_url = track['preview_url']
            df_temp = pd.DataFrame({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, index=[0])
            #tracks_data = tracks_data.append({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, ignore_index=True)
            tracks_data = pd.concat([tracks_data, df_temp], axis=0)
    # Print number of tracks found and write to CSV file
    print(f'Found {len(tracks_data)} tracks')
    tracks_data.to_csv('tracks_data_plus.csv', index=False)
    
    # Define search query
    query = f'acousticness:{feature*step_size:.3f}'

    # Search for tracks
    results = sp.search(q=query, type='track', market='US', limit=50)

    # Add new tracks to tracks_data DataFrame (if not already in DataFrame)
    for track in results['tracks']['items']:
        if track['id'] is not None and track['id'] not in tracks_data['track_id'].values:
            track_name = track['name']
            track_artist = track['artists'][0]['name']
            preview_url = track['preview_url']
            df_temp = pd.DataFrame({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, index=[0])
            #tracks_data = tracks_data.append({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, ignore_index=True)
            tracks_data = pd.concat([tracks_data, df_temp], axis=0)
    # Print number of tracks found and write to CSV file
    print(f'Found {len(tracks_data)} tracks')
    tracks_data.to_csv('tracks_data_plus.csv', index=False)
    
    # Define search query
    query = f'energy:{feature*step_size:.3f}'

    # Search for tracks
    results = sp.search(q=query, type='track', market='US', limit=50)

    # Add new tracks to tracks_data DataFrame (if not already in DataFrame)
    for track in results['tracks']['items']:
        if track['id'] is not None and track['id'] not in tracks_data['track_id'].values:
            track_name = track['name']
            track_artist = track['artists'][0]['name']
            preview_url = track['preview_url']
            df_temp = pd.DataFrame({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, index=[0])
            #tracks_data = tracks_data.append({'track_id': track_id, 'track_name': track_name, 'track_artist': track_artist, 'preview_url': preview_url}, ignore_index=True)
            tracks_data = pd.concat([tracks_data, df_temp], axis=0)
            
    # Print number of tracks found and write to CSV file
    print(f'Found {len(tracks_data)} tracks')
    tracks_data.to_csv('tracks_data_plus.csv', index=False)

working with feature interval: 0.999
Found 15371 tracks
Found 15371 tracks
Found 15372 tracks
Found 15372 tracks
Found 15373 tracks
working with feature interval: 0.998
Found 15382 tracks
Found 15382 tracks
Found 15384 tracks
Found 15384 tracks
Found 15384 tracks
working with feature interval: 0.997
Found 15385 tracks
Found 15385 tracks
Found 15386 tracks
Found 15386 tracks
Found 15386 tracks
working with feature interval: 0.996
Found 15386 tracks
Found 15386 tracks
Found 15387 tracks
Found 15387 tracks
Found 15387 tracks
working with feature interval: 0.995
Found 15393 tracks
Found 15393 tracks
Found 15394 tracks
Found 15399 tracks
Found 15405 tracks
working with feature interval: 0.994
Found 15405 tracks
Found 15405 tracks
Found 15406 tracks
Found 15406 tracks
Found 15406 tracks
working with feature interval: 0.993
Found 15411 tracks
Found 15411 tracks
Found 15412 tracks
Found 15412 tracks
Found 15412 tracks
working with feature interval: 0.992
Found 15415 tracks
Found 15415 tracks
F